In [1]:
from TransformerModel import TransformerModel
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
model = TransformerModel(2, 64,  1, 32, 32, 1)
model.to(device)

TransformerModel(
  (blocks): Sequential(
    (0): TransformerBlock(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0): MaskedSelfAttention(
            (key): Linear(in_features=64, out_features=64, bias=False)
            (query): Linear(in_features=64, out_features=64, bias=False)
            (value): Linear(in_features=64, out_features=64, bias=False)
          )
        )
        (proj): Linear(in_features=64, out_features=64, bias=True)
      )
      (ffwd): Sequential(
        (0): Linear(in_features=64, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=64, bias=True)
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
  )
  (ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=64, out_features=2, bias=True)
)

In [4]:
from torch.utils.data import DataLoader
from AudioDataset import AudioDataset

In [5]:
train_target_folder = "dataset-cutted/training/target"
train_non_target_folder = "dataset-cutted/training/non_target"

validation_target_folder = "dataset-cutted/validation/target"
validation_non_target_folder = "dataset-cutted/validation/non_target"

In [6]:
train_dataset = AudioDataset(train_target_folder, train_non_target_folder,fixed_length_spect= 700, train=True)
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)

validation_dataset = AudioDataset(validation_target_folder, validation_non_target_folder,fixed_length_spect = 700, train=False)
validation_dataloader = DataLoader(validation_dataset, batch_size = 32, shuffle = True)

In [7]:
from util import train_and_evaluate_vit

optimizer = torch.optim.AdamW(model.parameters(),lr=0.001)
epochs = 10

train_and_evaluate_vit(model, train_dataloader, validation_dataloader,  optimizer, epochs, device, checkpoint = None)

Lancement du training pour 10 epochs


100%|██████████| 353/353 [00:56<00:00,  6.28it/s]


Epoch 0: Training Loss = 0.21361082697303033, Validation Accuracy = 93.53% 
New best accuracy has been found
Saving model to model model.0.0.9352679252624512.pth


100%|██████████| 353/353 [00:54<00:00,  6.43it/s]


Epoch 1: Training Loss = 0.09828614418080905, Validation Accuracy = 94.42% 
New best accuracy has been found
Saving model to model model.1.0.9441964626312256.pth


...

Epoch 7: Training Loss = 0.045597970186239345, Validation Accuracy = 94.87% 


100%|██████████| 353/353 [00:55<00:00,  6.37it/s]


Epoch 8: Training Loss = 0.04317184471477103, Validation Accuracy = 94.20% 


100%|██████████| 353/353 [01:00<00:00,  5.88it/s]


Epoch 9: Training Loss = 0.03960693761457032, Validation Accuracy = 93.30% 


In [10]:
from util import evaluate_model_metrics 

evaluate_model_metrics(validation_dataloader, model, device)

14


{'True Positives': 137,
 'True Negatives': 281,
 'False Positives': 19,
 'False Negatives': 4,
 'Accuracy': 0.9478458049886621,
 'F1 Score': 0.9225589225589226,
 'F2 Score': 0.9513888888888888}

## Export the model

In [11]:
model.eval()
for p in model.parameters():
    p.requires_grad_(False)

input, target = train_dataset[0]
input = input.to(device)
traced = torch.jit.trace(model, input)
traced.save("traced_model.pth")
